In [1]:
import numpy as np
import tensorflow as tf
import codecs
import unidecode
import re

from random import shuffle
from nltk.corpus import stopwords
from googletrans import Translator

In [2]:
translator = Translator()

In [3]:
NUM_FEATURES = 50
SEQ_LEN = 250
CELL_SIZE = 128
BATCH_SIZE = 32
NUM_CLASESS = 3  

HIDDEN_LAYER_SIZE = 32

In [4]:
wordsList = np.load('wordsList.npy')
print('cargar el diccionario!')
wordsList = wordsList.tolist() #Originally loaded as numpy array
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('wordVectors.npy')
print ('cargar word vectors!')
print(len(wordsList))
print(wordVectors.shape)

cargar el diccionario!
cargar word vectors!
400000
(400000, 50)


In [5]:
sess = tf.Session()
tf.reset_default_graph()

labels = tf.placeholder(dtype = tf.float32, shape = [None, NUM_CLASESS]) # batch x classes 
input_data = tf.placeholder(dtype = tf.int32, shape = [None, SEQ_LEN]) # batch  x seq_len 

hin_f = tf.placeholder(dtype = tf.float32, shape = [None, CELL_SIZE]) # batch x cell
hin_b = tf.placeholder(dtype = tf.float32, shape = [None, CELL_SIZE]) # batch x cell

In [6]:
def my_attention(inputs, hidden_layer_size):
    X = tf.reshape(inputs, [-1, 2 * CELL_SIZE])
    Y = tf.layers.dense(X, hidden_layer_size, activation=tf.nn.relu)
    logits = tf.layers.dense(Y, 1, activation= None)
    logits = tf.reshape(logits, [-1, SEQ_LEN, 1])
    alphas = tf.nn.softmax(logits, dim=1)
    encoded_sentence = tf.reduce_sum(inputs * alphas, axis=1)
    
    return encoded_sentence, alphas

In [7]:
#data = tf.Variable(tf.zeros([None, SEQ_LEN, NUM_FEATURES]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

# creamos bidirectional gru
# layer 0
rnn_fw_cell_0 = tf.contrib.rnn.GRUCell(CELL_SIZE)
rnn_bw_cell_0 = tf.contrib.rnn.GRUCell(CELL_SIZE)
# layer 1
rnn_fw_cell_1 = tf.contrib.rnn.GRUCell(CELL_SIZE)
rnn_bw_cell_1 = tf.contrib.rnn.GRUCell(CELL_SIZE)

list_rnn_fw = [rnn_fw_cell_0, rnn_fw_cell_1]
list_rnn_bw = [rnn_bw_cell_0, rnn_bw_cell_1]

#outputs, _  = tf.nn.bidirectional_dynamic_rnn(cell_bw=rnn_bw_cell,cell_fw=rnn_fw_cell,inputs = data,dtype = tf.float32)
                                                #initial_state_bw=hin_b, initial_state_fw=hin_b)
outputs,_,_ = tf.contrib.rnn.stack_bidirectional_dynamic_rnn(cells_bw = list_rnn_fw, cells_fw = list_rnn_bw,
                                                             inputs = data, dtype=tf.float32)

#outputs = tf.concat(outputs, axis=2)

encoded, alphas = my_attention(outputs, HIDDEN_LAYER_SIZE)

logits = tf.layers.dense(encoded, 3, activation=None)
prediction = tf.argmax(logits, 1)

true_false = tf.equal(prediction, tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(true_false, tf.float32))

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits))
#loss = tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=logits)
optimizer = tf.train.AdamOptimizer().minimize(loss)

Instructions for updating:
dim is deprecated, use axis instead
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [8]:
# cargamos un modelo ya pre-entrenado
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('models2/'))

INFO:tensorflow:Restoring parameters from models2/model_GRU_Attention.ckpt-20000


In [9]:
def filter_text(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    return text

In [10]:
# reviews es una lista de reviews y polarities una lista de int
def get_format_data(reviews, polarities):
    arr = np.zeros((len(polarities), SEQ_LEN),dtype='uint32')
    arr_labels = []
    for r,review in enumerate(reviews):
        review = filter_text(unidecode.unidecode(review))
        review = review.split()
        for w, word in enumerate(review):
            if w >= SEQ_LEN: break
            try:
                arr[r][w] = wordsList.index(word)
            except ValueError:
                arr[r][w] = 399999
    for l in polarities:
        if l == 0: arr_labels.append([1,0,0])
        if l == 1: arr_labels.append([0,1,0])
        if l == 2: arr_labels.append([0,0,1])
    return arr, arr_labels

In [10]:
filename1 = 'test/intertass-test-2017.csv'
text1 = codecs.open(filename1, encoding='utf-8')
next(text1)

sentences1 = []
polarity1 = []

i = 0
for line in text1:
    lista = line.split(',') # asumimos que el review no contiene comas 
    w = str(lista[1]).split('\n')[0]
    if w == 'NONE' or w == 'NEU': polarity1.append(2) # neutro
    if w == 'P': polarity1.append(1) # positive
    if w == 'N': polarity1.append(0)   # negative
    sentence = lista[0]
    #sentence = filter_text(unidecode.unidecode(sentence))
    sentences1.append(sentence)
    i += 1
print i

NUM_EJEM_1 = len(sentences1)

1899


In [11]:
a, b = get_format_data(sentences1, polarity1)

In [14]:
p = sess.run(accuracy, {input_data: a, labels: b})
print (p*100)
del a
del b

57.8199028968811


In [10]:
# read the tass con indexs

In [11]:
a = np.load('idsMatrix_2.npy')
b = np.load('labels_2.npy')
b = b.tolist()

bb = []
for l in b:
    if l == 0: bb.append([1,0,0])
    if l == 1: bb.append([0,1,0])
    if l == 2: bb.append([0,0,1])

In [12]:
p = sess.run(accuracy, {input_data: a, labels: bb})
print (p*100)

63.7704074382782


In [43]:
proof = ['Esta pelicula no me gusto nada', 'me gusta jugar futbol, aunque pierda']
t_sentences_es = translator.translate(proof, dest='en')
proof_en = [t.text for t in t_sentences_es]

proof = np.array(proof)
proof_b = [0,1]

a , b = get_format_data(proof_en, proof_b)

In [44]:
p = sess.run(prediction, {input_data: a})

In [45]:
p

array([0, 2])